# Forecast

Based on our current scouting, as well as The Blue Alliance's OPR score, and any other criteria we have, guess the outcome of the matches to be played, and display the matches
that have already been played using the same criteria

*run all Data notebooks before any of the analysis notebooks.*

In [8]:
import bokeh
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.io import output_notebook, show

import glob
import logging

import numpy as np
import pandas as pd

import sys

from common import canalytics

output_notebook()
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger('Forecast_Analysis')

Loading BokehJS ...

In [24]:
# Without index_col=0, pandas will create a new, made up index for each line read from the csv.
tba_matches = pd.read_csv('./data/tba_matches.csv',index_col=0)
team_summary = pd.read_csv('./data/team_summary.csv',index_col=0)
team_summary.set_index('team',inplace=True)


In [25]:
team_summary.head()


,matches,scouted_matches,scouting_points_avg,climb_success_pct,climb_failure_pct,total_cargo_avg,auto_cargo_avg,teleop_cargo_avg,climb_points_avg,highest_endgame_position,oprs,pre_oprs,pre_climb_points,pre_district_rank,pre_district_points
team,,,,,,,,,,,,,,,
457,12,8,5.000000,16.666667,37.500000,1.250000,0.125,1.125000,1.000000,Mid,10.118089,10.12,10.83,84.0,24.0
2158,12,10,29.600000,50.000000,10.000000,10.500000,1.700,8.800000,3.000000,Mid,27.186538,27.19,8.25,55.0,33.0
2468,12,11,27.181818,91.666667,0.000000,7.818182,1.000,6.818182,10.916667,Traversal,29.282110,NaN,NaN,NaN,NaN
2583,12,6,1.500000,8.333333,16.666667,0.833333,0.000,0.833333,0.500000,Mid,7.418845,NaN,NaN,NaN,NaN
2687,12,10,25.000000,91.666667,0.000000,6.500000,0.700,5.800000,9.000000,Traversal,31.423240,NaN,NaN,NaN,NaN


In [26]:
tba_matches.head()

,match_number,post_result_time,predicted_time,red1,red2,red3,red1_taxi,red2_taxi,red3_taxi,red1_climb,...,blue1_climb,blue2_climb,blue3_climb,blue_points,blue_fouls,blue_techfouls,blue_teleop_cargo,blue_auto_cargo,blue_teleop_low,blue_teleop_high
0,1,2022-03-18 11:08,2022-03-18 11:03,5908,5726,8819,Yes,Yes,No,None,...,Traversal,High,None,75,0,0,14,4,0,14
11,2,2022-03-18 11:23,2022-03-18 11:20,8369,8088,7750,No,Yes,No,None,...,Mid,None,None,35,1,0,19,2,19,0
22,3,2022-03-18 11:29,2022-03-18 11:30,2687,8573,8114,Yes,No,Yes,High,...,Mid,Traversal,None,32,2,0,4,0,1,3
33,4,2022-03-18 11:37,2022-03-18 11:35,3561,8144,7088,Yes,No,Yes,None,...,None,Traversal,None,41,0,0,8,1,2,6
44,5,2022-03-18 11:49,2022-03-18 11:45,5986,8879,2789,No,Yes,Yes,None,...,None,None,Mid,34,1,0,8,2,0,8


In [27]:

tba_matches['oprs_forecast_red']=tba_matches.apply(lambda row: team_summary.at[row.red1,'oprs']+team_summary.at[row.red2,'oprs']+team_summary.at[row.red3,'oprs'], axis = 1 )
tba_matches['oprs_forecast_blue']=tba_matches.apply(lambda row: team_summary.at[row.blue1,'oprs']+team_summary.at[row.blue2,'oprs']+team_summary.at[row.blue3,'oprs'], axis = 1 )
tba_matches['scouting_forecast_red']=tba_matches.apply(lambda row: team_summary.at[row.red1,'scouting_points_avg']+
                                                       team_summary.at[row.red2,'scouting_points_avg']+
                                                       team_summary.at[row.red3,'scouting_points_avg'], axis = 1 )
tba_matches['scouting_forecast_blue']=tba_matches.apply(lambda row: team_summary.at[row.blue1,'scouting_points_avg']+
                                                        team_summary.at[row.blue2,'scouting_points_avg']+
                                                        team_summary.at[row.blue3,'scouting_points_avg'], axis = 1 )
# Remove all columns outside of what we are using to put in the table.
tba_matches=tba_matches[['match_number','post_result_time','red1','red2','red3','blue1','blue2','blue3',
                         'red_points','blue_points','oprs_forecast_red','oprs_forecast_blue','scouting_forecast_red','scouting_forecast_blue']]

# This rounds across all columns and rows in the dataframe, and then sorts the dataframe to ensure it is in match order.
tba_matches=tba_matches.round()
tba_matches.sort_values(by='match_number',inplace=True)

In [28]:
source = ColumnDataSource(tba_matches)

columns = [
        TableColumn(field="match_number", title="Match"),
        TableColumn(field="red1", title="Red 1"),
        TableColumn(field="red2", title="Red 2"),
        TableColumn(field="red3", title="Red 3"),
        TableColumn(field="blue1", title="Blue 1"),
        TableColumn(field="blue2", title="Blue 2"),
        TableColumn(field="blue3", title="Blue 3"),
        TableColumn(field="red_points", title="Red Actual"),
        TableColumn(field="blue_points", title="Blue Actual"),
        TableColumn(field="oprs_forecast_red", title="OPRS Red"),
        TableColumn(field="oprs_forecast_blue", title="OPRS Blue"),
        TableColumn(field="scouting_forecast_red", title="Scout Red"),
        TableColumn(field="scouting_forecast_blue", title="Scout Blue")]
    
    
    
forecast_table = DataTable(source=source, columns=columns, width=1000,height=1200,
        sortable=True,
        editable=False,
        fit_columns=True,
        reorderable=True)

# Forecasting 

The table shows all of the current qualifying matches for the event.  If the event has already occurred, it will show the Actual score for both Red and Blue teams.

In addition it will also show two forecasts for each match.  One is based on OPRS from The Blue Alliance, and the other is average points based on the scouting app input.




In [29]:
show(forecast_table)